# Random walk

In [2]:
#| default_exp graph.operations

In [76]:
#| export
import scipy.sparse as sp, numpy as np

from typing import Optional, List
from tqdm.auto import tqdm
from functools import partial
from xclib.utils.sparse import retain_topk

In [77]:
#| export
class Graph:

    @staticmethod
    def batched_random_walk(
        data_lbl_indptr:List,
        data_lbl_indices:List,
        lbl_data_indptr:List,
        lbl_data_indices:List,
        data_start:int, 
        data_end:int,
        n_lbl:int,
        walk_to:Optional[int]=100, 
        prob_reset:Optional[float]=0.8, 
        n_hops:Optional[int]=1,
    ):
        n_data = data_end - data_start
        
        nbr_indices = np.zeros((n_data, walk_to), dtype=np.int32)
        nbr_data = np.zeros((n_data, walk_to), dtype=np.float32)
        for i in range(n_data):
            data_i = i + data_start
            for j in range(walk_to):
                p,q = data_lbl_indptr[data_i], data_lbl_indptr[data_i+1]
                if p == q: continue
                lbl_i = np.random.choice(data_lbl_indices[p:q])
                
                p,q = lbl_data_indptr[lbl_i], lbl_data_indptr[lbl_i+1]
                if p == q: continue
                data_i = np.random.choice(lbl_data_indices[p:q])
        
                nbr_indices[i, j] = data_i if n_hops == 2 else lbl_i
                nbr_data[i, j] = 1.0
                if np.random.random() < prob_reset: data_i = i + data_start
                
        nbr_indptr = np.arange(0, walk_to*(n_data+1), walk_to)
        
        matrix = sp.csr_matrix((nbr_data.flatten(), nbr_indices.flatten(), nbr_indptr), shape=(n_data, n_lbl), dtype=np.float32)
        matrix.eliminate_zeros()
        matrix.sum_duplicates()
        matrix.sort_indices()
    
        return matrix

    @staticmethod
    def random_walk(
        data_lbl:sp.csr_matrix, 
        walk_to:Optional[int]=100, 
        batch_size:Optional[int]=1024, 
        prob_reset:Optional[float]=0.8, 
        n_hops:Optional[int]=1,
    ):
        lbl_data = data_lbl.transpose()
        lbl_data = lbl_data.tocsr()
    
        data_lbl_indptr, data_lbl_indices = data_lbl.indptr, data_lbl.indices
        lbl_data_indptr, lbl_data_indices = lbl_data.indptr, lbl_data.indices
    
        n_lbl = data_lbl.shape[0] if n_hops == 2 else data_lbl.shape[1]
    
        walk_func = partial(Graph.batched_random_walk, data_lbl_indptr=data_lbl_indptr, data_lbl_indices=data_lbl_indices, 
                            lbl_data_indptr=lbl_data_indptr, lbl_data_indices=lbl_data_indices, walk_to=walk_to, prob_reset=prob_reset, 
                            n_hops=n_hops, n_lbl=n_lbl)
        
        return sp.vstack([walk_func(data_start=i, data_end=min(i+batch_size, data_lbl.shape[0])) for i in tqdm(range(0, data_lbl.shape[0], batch_size))])

    @staticmethod
    def one_hop_matrix(data_lbl:sp.csr_matrix, batch_size:int=1024, topk:Optional[int]=None):
        data_lbl_t = data_lbl.transpose().tocsr()
        lbl_lbl = sp.vstack([data_lbl_t[i:i+batch_size]@data_lbl for i in tqdm(range(0, data_lbl_t.shape[0], batch_size))])
        data_lbl = sp.vstack([data_lbl[i:i+batch_size]@lbl_lbl for i in tqdm(range(0, data_lbl.shape[0], batch_size))])
        lbl_lbl.sort_indices()
        data_lbl.sort_indices()
        if topk is not None:
            data_lbl, lbl_lbl = retain_topk(data_lbl, k=topk), retain_topk(lbl_lbl, k=topk)
        return data_lbl, lbl_lbl

    @staticmethod
    def threshold_on_degree(data_lbl:sp.csr_matrix, thresh:int=10):
        data_lbl = data_lbl.copy()
        idx = np.where(data_lbl.getnnz(axis=1) > thresh)[0]
        for i in idx:
            p,q = data_lbl.indptr[i],data_lbl.indptr[i+1]
            data_lbl.data[p:q] = 0
        data_lbl.eliminate_zeros()
        return data_lbl

    @staticmethod
    def normalize(mat):
        col_nnz = np.sqrt(1/(mat.sum(axis=0) + 1e-6))
        row_nnz = np.sqrt(1/(mat.sum(axis=1)+ 1e-6))
        mat = r_diags.multiply(row_nnz).multiply(col_nnz)
        mat.tocsr(); mat.eliminate_zeros()
        return mat
        

In [98]:
#| export
def perform_random_walk(
    data_lbl:sp.csr_matrix, 
    batch_size:Optional[int]=1024, 
    walk_to:Optional[int]=100, 
    prob_reset:Optional[float]=0.8, 
    n_hops:Optional[int]=2,
    thresh:Optional[int]=10,
):
    data_lbl = Graph.threshold_on_degree(data_lbl, thresh=thresh)
    data_lbl = Graph.random_walk(data_lbl, walk_to=walk_to, batch_size=batch_size, prob_reset=prob_reset, n_hops=n_hops)
    data_lbl = data_lbl / (data_lbl.sum(axis=1) + 1e-9)
    data_lbl = data_lbl.tocsr()
    return data_lbl
    

## `Example`

In [ ]:
from xcai.basics import *

In [ ]:
pickle_dir = '/home/scai/phd/aiz218323/scratch/datasets/processed'
use_sxc_sampler = True

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

pkl_file = f'{pickle_dir}/cachew/wikiseealsotitles_data-meta_distilbert-base-uncased'
pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
pkl_file = f'{pkl_file}.joblib'

In [ ]:
block = build_block(pkl_file, config_file, use_sxc_sampler, config_key, data_dir=data_dir)

In [101]:
batch_size = 1024
walk_to = 100
prob_reset = 0.8
n_hops = 1
thresh = 10

In [102]:
data_lbl = block.train.dset.data.data_lbl

In [103]:
matrix = perform_random_walk(data_lbl, batch_size=batch_size, walk_to=walk_to, prob_reset=prob_reset, n_hops=n_hops, thresh=thresh)

  0%|          | 0/677 [00:00<?, ?it/s]

In [104]:
matrix

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3830241 stored elements and shape (693082, 312330)>